# Analisa informações gerais dos arquivos de Twitter #

In [ ]:
#Arquivo que analisa o volume de tweets de cada série
# Bibliotecas
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
df_arquivo = pd.read_csv('series_dataset/SexEducation.csv')
df_arquivo2 = pd.read_csv('series_dataset/Ozark.csv')


In [ ]:
df_arquivo['date'].head(5)
df_arquivo.dtypes

df_arquivo['date'] = pd.to_datetime(df_arquivo['date'], format="%Y-%m-%d", errors = 'coerce')
df_arquivo2['date'] = pd.to_datetime(df_arquivo2['date'], format="%Y-%m-%d", errors = 'coerce')

y1 = df_arquivo['date'].value_counts()
x1 = df_arquivo['date'].value_counts().index.tolist()

y2 = df_arquivo2['date'].value_counts()
x2 = df_arquivo2['date'].value_counts().index.tolist()



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9.8, 5), sharey=False)

sns.lineplot(ax=axes[0], x = x1, y = y1,data = df_arquivo['date'], color="salmon")
axes[0].tick_params(rotation=90, axis='x')
axes[0].set_title('Número Total de Twitts por Dia')
axes[0].set_ylabel('Número Total de Twitts')
axes[0].set_xlabel('Dia')

sns.lineplot(ax=axes[1], x = x2, y = y2,data = df_arquivo2['date'], color="salmon")
axes[1].tick_params(rotation=90, axis='x')
axes[1].set_title('Número Total de Twitts por Dia')
axes[1].set_ylabel('Número Total de Twitts')
axes[1].set_xlabel('Dia')



